# CS 282
### Programming Assignment 1
#### Item 2

Jan Lendl R. Uy

2019-00312

In [5]:
!pip install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import cv2
import numpy as np
import os.path

def main():
    # Prompt for video filename
    video_path = input("Enter video filename: ")
    
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' not found")
        return
    
    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return
    
    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 30  # Default if invalid
    
    # Always ask about orientation for all videos
    print(f"Video dimensions: {width}x{height}")
    response = input("Fix orientation by rotating 90 degrees clockwise? (y/n): ").lower()
    fix_orientation = response.startswith('y')
    
    # Global variables to track state
    paused = False
    
    # Create window with enough space for video and controls
    window_name = 'Video Player'
    cv2.namedWindow(window_name)
    
    # Function to process and display a frame with controls
    def display_frame_with_controls(frame, is_paused):
        # Fix orientation if needed
        if fix_orientation:
            # Rotate the frame (90 degrees clockwise)
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        
        # Get dimensions after rotation
        frame_height, frame_width = frame.shape[:2]
        
        # Create a larger canvas with space for controls
        controls_height = 60
        canvas = np.ones((frame_height + controls_height, frame_width, 3), dtype=np.uint8) * 240  # Light gray
        
        # Add video frame to canvas
        canvas[0:frame_height, 0:frame_width] = frame
        
        # Draw separator line
        cv2.line(canvas, (0, frame_height), (frame_width, frame_height), (180, 180, 180), 2)
        
        # Button dimensions
        button_width = 100
        button_height = 40
        button_x = 20
        button_y = frame_height + 10
        
        # Draw pause/play button
        label = "PAUSE" if not is_paused else "PLAY"
        button_color = (120, 120, 120) if is_paused else (0, 120, 0)  # Gray when paused, green when playing
        
        cv2.rectangle(canvas, 
                     (button_x, button_y), 
                     (button_x + button_width, button_y + button_height), 
                     button_color, 
                     -1)  # Filled rectangle
        
        # Add button text
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        text_x = button_x + (button_width - text_size[0]) // 2
        text_y = button_y + (button_height + text_size[1]) // 2
        cv2.putText(canvas, label, (text_x, text_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Add slider label
        cv2.putText(canvas, "Position:", (button_x + button_width + 20, button_y + 25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
        
        # Display the combined canvas
        cv2.imshow(window_name, canvas)
        
        return canvas, button_x, button_y, button_width, button_height
    
    # Function to handle slider changes
    def on_slider_change(pos):
        frame_pos = int((pos / 10.0) * total_frames)
        if frame_pos >= total_frames:
            frame_pos = total_frames - 1
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
        ret, frame = cap.read()
        if ret:
            display_frame_with_controls(frame, paused)
    
    # Create slider
    cv2.createTrackbar('Position', window_name, 0, 10, on_slider_change)
    
    # Variables to track button position
    current_button_x = 20
    current_button_y = height + 10 if not fix_orientation else width + 10
    current_button_width = 100
    current_button_height = 40
    
    # Mouse callback function to handle button clicks
    def mouse_callback(event, x, y, flags, param):
        nonlocal paused, current_button_x, current_button_y, current_button_width, current_button_height
        
        if event == cv2.EVENT_LBUTTONDOWN:
            # Check if click is within button area
            if (current_button_x <= x <= current_button_x + current_button_width and 
                current_button_y <= y <= current_button_y + current_button_height):
                # Toggle pause state
                paused = not paused
                print("Video", "paused" if paused else "playing")
                
                # Update the display with current frame
                current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                # Go back one frame to stay at current position
                if current_frame > 0:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame - 1)
                ret, frame = cap.read()
                if ret:
                    # Update button positions from display function
                    canvas, current_button_x, current_button_y, current_button_width, current_button_height = display_frame_with_controls(frame, paused)
    
    # Set mouse callback
    cv2.setMouseCallback(window_name, mouse_callback)
    
    # Read first frame
    ret, frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return
    
    # Show first frame with controls
    canvas, current_button_x, current_button_y, current_button_width, current_button_height = display_frame_with_controls(frame, paused)
    
    # Print instructions
    print("\nControls:")
    print("- Use the 'Position' slider to navigate the video (0-10)")
    print("- Click the PAUSE/PLAY button to control playback")
    print("- Press 'q' or 'ESC' to quit")
    print("- Press SPACE to toggle pause/play")
    
    current_pos = 0
    
    # Main loop
    while True:
        # Handle key presses (5ms wait for responsiveness)
        key = cv2.waitKey(5) & 0xFF
        if key == 27 or key == ord('q'):
            break
        
        # Space bar can also toggle pause
        if key == 32:  # Space key
            paused = not paused
            print("Video", "paused" if paused else "playing")
            
            # Update the display
            current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            # Go back one frame to stay at current position
            if current_frame > 0:
                cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame - 1)
            ret, frame = cap.read()
            if ret:
                canvas, current_button_x, current_button_y, current_button_width, current_button_height = display_frame_with_controls(frame, paused)
        
        # If not paused, play video
        if not paused:
            ret, frame = cap.read()
            
            # If reached end of video, loop back
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                ret, frame = cap.read()
                current_pos = 0
            
            if ret:
                # Show the frame with controls and update button positions
                canvas, current_button_x, current_button_y, current_button_width, current_button_height = display_frame_with_controls(frame, paused)
                
                # Update position
                current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                new_pos = int((current_frame / total_frames) * 10)
                
                # Only update trackbar if position changed
                if new_pos != current_pos:
                    current_pos = new_pos
                    cv2.setTrackbarPos('Position', window_name, current_pos)
                
                # Control playback speed
                cv2.waitKey(int(1000/fps))
    
    # Clean up
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Video dimensions: 1920x1080

Controls:
- Use the 'Position' slider to navigate the video (0-10)
- Click the PAUSE/PLAY button to control playback
- Press 'q' or 'ESC' to quit
- Press SPACE to toggle pause/play
Video paused
